<a href="https://colab.research.google.com/github/raviteja777/neural_network_demos/blob/master/pytorch_cnn_cifar_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision import transforms as transforms


https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py


#### Steps to follow

1. Load and normalizing the CIFAR10 training and test datasets using torchvision
2. Define a convolutional neural network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

In [ ]:
# step 1 - load and transform cifar data  

# define a transform for the data 
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])

#load the train data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,shuffle=True,num_workers=2)

#load the test data 
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# display some sample training images

import numpy as np
import matplotlib.pyplot as plt


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)

#dataiter.iter loads next 4 images ??
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))

# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))



In [ ]:
### define a convolution neural network

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
### Tip : how to decide the layer input output size -- FOR INFORMATION ONLY 
# for conv layers (num_inp_channels, num_out_channels , kernal_size)  -- kernal_size = 5 specifies=> 5*5 filter

c1 = nn.Conv2d(3,6,5)
c2 = nn.Conv2d(6,16,5)
pool = nn.MaxPool2d(2,2)

# take first sample from dataloader
rec,res = trainloader.dataset[0]  

# data point is (3,32,32) - i.e, 3 channels , 32*32 pixels image dimensions
# after convolution the size might reduce because of padding settings - information on the border might be lost
# convert into 4-d array (1st param is batch size - it is 1 here since only 1 record) 
# tensor.view changes dimension 
rec = rec.view(1,3,32,32)  
rec = pool(F.relu(c1(rec)))
print(rec.shape)
rec = pool(F.relu(c2(rec)))
print(rec.shape)



In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)
  
  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

cnn = CNN()


In [ ]:
cnn

In [ ]:
### Define a loss function and optimizer

import torch.optim as optim

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=cnn.parameters(),lr=0.01)

In [ ]:
### train the network

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')



In [ ]:
#save the model 

PATH = './cifar_net.pth'
torch.save(cnn.state_dict(), PATH)

In [ ]:
#test the model

dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
cnn = CNN()
cnn.load_state_dict(torch.load(PATH))
ouputs = cnn(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
# Accuracy of model on test dataset 

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
